In [1]:
import os

In [2]:
os.chdir('..')

In [3]:
%pwd

'c:\\Users\\Kushal\\Documents\\GitHub\\kidney-disease-classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class BaseModelConfig:
    root_dir: Path
    base_model_path: Path
    params_num_epochs: int
    params_image_size: list
    params_learning_rate: float
    params_weight: str
    params_classes: int


In [5]:
from classifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from classifier.utils.common import read_yaml, create_directories


import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms

In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self)-> BaseModelConfig:
        config = self.config.prepare_base_model
        create_directories([config.root_dir])

        prepare_base_model_config = BaseModelConfig(
            root_dir= Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            params_num_epochs= self.params.EPOCHS,
            params_image_size = self.params.IMAGE_SIZE,
            params_learning_rate = self.params.LEARNING_RATE,
            params_weight = self.params.WEIGHTS,
            params_classes = self.params.CLASSES
        )

        return prepare_base_model_config

In [15]:
class PrepareBaseModel:
    def __init__(self, config: BaseModelConfig):
        self.config = config

    def get_base_model(self):
        model = models.resnet18(weights=self.config.params_weight)
        optimizer = torch.optim.Adam(model.parameters(), lr = self.config.params_learning_rate)
        for param in model.parameters():
            param.requires_grad = False
        num_ftrs = model.fc.in_features
        # Here the size of each output sample is set to 2.
        model.fc = nn.Linear(num_ftrs, 2)
        self.model = model
        self.optimizer = optimizer

        self.save_model()


    def save_model(self):
        # create_directories([self.config.base_model_path])
        torch.save({
            'epoch': 0,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict()

            }, self.config.base_model_path)

In [16]:
cm = ConfigurationManager()
bmc = cm.get_prepare_base_model_config()
# prep_base_model = PrepareBaseModel()

[2024-08-01 15:22:46,463: INFO: common yaml file: config\config.yaml loaded successfully]
[2024-08-01 15:22:46,466: INFO: common yaml file: params.yaml loaded successfully]
[2024-08-01 15:22:46,467: INFO: common created directory at: artifacts]
[2024-08-01 15:22:46,468: INFO: common created directory at: artifacts/prepare_base_model]


In [17]:
pbm = PrepareBaseModel(bmc)
pbm.get_base_model()